## **範例：多重 AI 代理人協助預訂酒店**

在現今快節奏的世界中，規劃商務旅行不僅僅是預訂機票和酒店房間。它需要一種協調和效率的水平，而這往往是很難達到的。這時，多重 AI 代理人就能派上用場，徹底改變我們管理旅行需求的方式。

想像一下，有一個由智能代理人組成的團隊隨時為您服務，協同處理旅行的每個細節，精準且輕鬆。憑藉我們的先進 AI 技術，我們創建了專門的代理人來負責預訂服務和行程安排，確保您擁有無縫且無壓力的旅行體驗。

這是一個基本場景。在規劃商務旅行時，我們需要與商務旅行代理人協商以獲取機票信息、酒店信息等。通過 AI 代理人，我們可以建立負責預訂服務的代理人以及負責行程安排的代理人，讓它們協同合作並提升智能化水平。


# 初始化 Azure AI Agent Service 並從 **.env** 獲取配置信息

### **.env**

建立一個 .env 文件

**.env** 包含 Azure AI Agent Service 的連接字串、AOAI 使用的模型，以及對應的 Google API 搜索服務 API、ENDPOINT 等。

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "您的 Azure AI Agent Service 模型部署名稱"

[**NOTE**] 您需要一個具有 100,000 速率限制（每分鐘的 Token 數）和 600 速率限制（每分鐘請求數）的模型。

  您可以在 Azure AI Foundry 的模型與端點中獲取模型。

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "您的 Azure AI Agent Service 專案連接字串"

  您可以在 AI Foundry Portal 的專案概覽頁面中獲取專案連接字串。

- **SERPAPI_SEARCH_API_KEY** = "您的 SERPAPI 搜索 API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "您的 SERPAPI 搜索 ENDPOINT"

要獲取 Azure AI Agent Service 的模型部署名稱和專案連接字串，您需要創建 Azure AI Agent Service。建議直接使用[此模板](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json)來創建。（***注意：*** Azure AI Agent Service 目前僅限於特定區域。建議參考[此連結](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support)來設置區域）

Agent 需要訪問 SERPAPI。建議使用[此連結](https://serpapi.com/searches)進行註冊。註冊後，您可以獲取唯一的 API KEY 和 ENDPOINT。


# 登錄 Azure

您現在需要登錄到 Azure。請在 VScode 中打開終端並執行 `az login` 命令。


# 設置

要執行此筆記本，您需要安裝以下庫。以下是所需庫及其對應的 pip 安裝命令清單：

azure-identity：用於 Azure 身份驗證。  
requests：用於發送 HTTP 請求。  
semantic-kernel：用於語義核心框架（假設這是一個自定義或特定的庫，您可能需要從特定來源或存儲庫安裝它）。  


In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli

# 解釋：
import asyncio：這是匯入 asyncio 模組，該模組提供了 Python 中的非同步程式設計支援。它允許你使用 async 和 await 語法撰寫並行程式碼。
from typing import Annotated：這是從 typing 模組匯入 Annotated 類型。Annotated 用於為型別提示添加元數據，這在驗證、文件或工具等多種用途中可能非常有用。


In [ ]:
import asyncio,os
from typing import Annotated

# 說明：
透過使用 `from dotenv import load_dotenv` 和 `load_dotenv()`，您可以輕鬆地在 `.env` 文件中管理配置設定和敏感資訊（例如 API 金鑰和資料庫 URL），將它們與源代碼分離，從而使您的應用程式更安全且更易於配置。


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# 說明：

匯入語句：from azure.identity.aio import DefaultAzureCredential：這是從 azure.identity.aio 模組中匯入 DefaultAzureCredential 類別。模組名稱中的 aio 表示它是為非同步操作設計的。

DefaultAzureCredential 的用途：DefaultAzureCredential 類別是 Azure SDK for Python 的一部分。它提供了一種預設的方式來驗證 Azure 服務。它會按照特定順序嘗試使用多種方法進行驗證，例如環境變數、受管理的身分識別以及 Azure CLI 憑證。

非同步操作：aio 模組表示 DefaultAzureCredential 類別支援非同步操作。這意味著您可以使用 asyncio 來執行非阻塞的驗證請求。


In [ ]:
from azure.identity.aio import DefaultAzureCredential

# 說明：
從 `semantic_kernel` 套件中匯入各種模組和類別。以下是每個匯入項目的詳細說明：

AgentGroupChat 從 `semantic_kernel.agents` 匯入：  
此類別負責處理與 AI 代理相關的群組聊天功能。

AzureAIAgent 和 AzureAIAgentSettings 從 `semantic_kernel.agents.azure_ai` 匯入：  
- **AzureAIAgent**：此類別用於建立和管理利用 Azure AI 服務的 AI 代理。  
- **AzureAIAgentSettings**：此類別用於配置 AzureAIAgent 的設定。

TerminationStrategy 從 `semantic_kernel.agents.strategies.termination.termination_strategy` 匯入：  
此類別定義了在特定條件下終止 AI 代理執行的策略。

ChatMessageContent 從 `semantic_kernel.contents.chat_message_content` 匯入：  
此類別用於處理聊天訊息的內容。

AuthorRole 從 `semantic_kernel.contents.utils.author_role` 匯入：  
此類別定義了在聊天訊息上下文中作者的不同角色。

kernel_function 從 `semantic_kernel.functions.kernel_function_decorator` 匯入：  
此裝飾器用於定義核心函數，這些函數可以在 `semantic_kernel` 框架內執行。

這些匯入項目設置了必要的組件，用於建立和管理能夠在群組聊天環境中互動的 AI 代理，可能用於像預訂酒店等任務。


In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# 說明：
接下來，我們從 azure.ai.projects.models 模組中匯入 CodeInterpreterTool 類別。

CodeInterpreterTool：此類別是 Azure AI SDK 的一部分，用於在 AI 專案的上下文中解釋和執行程式碼。它提供了執行程式碼片段、分析程式碼或將程式碼執行整合到 AI 工作流程中的功能。
此匯入設定了使用 CodeInterpreterTool 的必要元件，這在需要動態解釋和執行程式碼的任務中可能非常有用。


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# 說明：
ApprovalTerminationStrategy 類別提供了一種特定策略，用於終止 AI 代理的運作。如果其互動歷史中的最後一則訊息包含「saved」這個詞，代理就會終止運作。在某些情境中，這可能非常有用，例如當代理的任務在收到某項已「保存」的確認後被視為完成時。定義互動方法。當預約計劃保存後，接收到保存信號時即可停止。


In [ ]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "saved" in history[-1].content.lower()

# 說明：

這行程式碼透過呼叫 create() 方法，初始化一個 AzureAIAgentSettings 物件，並使用預設或預先定義的設定。此設定物件 (ai_agent_settings) 隨後可用於配置和管理 AzureAIAgent 實例。


In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()

# 說明：
透過匯入 requests 函式庫，您可以輕鬆地在 Python 程式碼中發送 HTTP 請求並與網路服務進行互動。


In [ ]:
import requests

# 說明：
這是一個用來存儲 API 金鑰的變數，該金鑰用於訪問 SERP（搜尋引擎結果頁面）API 服務。API 金鑰是一個用於驗證與您帳戶相關的請求的唯一識別碼。

'GOOGLE_SEARCH_API_KEY'：這是一個佔位符字串。您需要將 'GOOGLE_SEARCH_API_KEY' 替換為您的實際 SERP API 金鑰。

目的：這行的目的是將 API 金鑰存儲在變數中，以便用於驗證對 SERP API 服務的請求。API 金鑰是訪問服務並執行搜尋所必需的。

如何獲取 SERP API 金鑰：要獲取 SERP API 金鑰，請按照以下一般步驟操作：https://serpapi.com（具體步驟可能因您使用的 SERP API 服務而有所不同）：

選擇一個 SERP API 服務：目前有多種 SERP API 服務可供選擇，例如 SerpAPI、Google Custom Search JSON API 等。選擇最符合您需求的服務。

註冊帳戶：

前往所選 SERP API 服務的網站 https://www.serpapi.com，註冊一個帳戶。您可能需要提供一些基本信息並驗證您的電子郵件地址。

創建 API 金鑰：

註冊完成後，登錄您的帳戶並導航到 API 區域或儀表板。尋找創建或生成新 API 金鑰的選項。
複製 API 金鑰：

API 金鑰生成後，請複製它。此金鑰將用於驗證您對 SERP API 服務的請求。
替換佔位符：

將佔位符替換到您的 .env 文件中。


In [ ]:
SERPAPI_SEARCH_API_KEY=os.getenv('SERPAPI_SEARCH_API_KEY')

In [ ]:
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT')

# 說明：
BookingPlugin 類別提供使用 Serpapi.com 的 Google 搜尋 API 來預訂酒店和航班的方法。它負責構建必要的參數、發送 API 請求，並處理回應以返回相關的預訂資訊。API 金鑰 (SERPAPI_SEARCH_API_KEY) 和端點 (SERPAPI_SEARCH_ENDPOINT) 用於驗證並向 Google 搜尋 API 發送請求。


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""
    @kernel_function(description="booking hotel")
    def booking_hotel(self,query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-in Time"])-> Annotated[str, "Return the result of booking hotel infomation"]:

        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY
        }

        response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params)
        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None

    
    @kernel_function(description="booking fight")
    def  booking_fight(self,origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"])-> Annotated[str, "Return the result of booking fight infomation"]:
        
        go_params = {
            "engine": "google_flights",   
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }

        print(go_params)

        go_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=go_params)


        result = ''

        if go_response.status_code == 200:
            response = go_response.json()

            result += "# outbound \n " + str(response)
        else:
            print('error!!!')
            # return None

        
        back_params = {
            "engine": "google_flights",   
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }


        print(back_params)


        back_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=back_params)



        if back_response.status_code == 200:
            response = back_response.json()

            result += "\n # return \n"  + str(response)

        else:
            print('error!!!')
            # return None
        
        print(result)

        return result

        


# 說明：
SavePlugin 類別提供了一個名為 saving_plan 的方法，用於使用 Azure AI 服務保存旅行計劃。它設置了 Azure 憑據，創建了一個 AI 代理，處理用戶輸入以生成並保存旅行計劃內容，並負責文件保存和清理操作。該方法在成功完成後返回 "Saved"。


In [ ]:
class SavePlugin:
    """Save Plugin for customers"""
    @kernel_function(description="saving plan")
    async def saving_plan(self,tripplan: Annotated[str, "The content of trip plan"])-> Annotated[str, "Return status of save content"]:

        async with (
            DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
        ):

            code_interpreter = CodeInterpreterTool()
            
            agent_definition = await client.agents.create_agent(
                model=ai_agent_settings.model_deployment_name,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )


            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
            )

            thread = await client.agents.create_thread()


            user_inputs = [
                """
            
                        You are my Python programming assistant. Generate code,save """+ tripplan +
                        
                    """    
                        and execute it according to the following requirements

                        1. Save blog content to trip-{YYMMDDHHMMSS}.md

                        2. give me the download this file link
                    """
            ]



            try:
                for user_input in user_inputs:
                    # Add the user input as a chat message
                    await agent.add_chat_message(
                        thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
                    )
                    print(f"# User: '{user_input}'")
                    # Invoke the agent for the specified thread
                    async for content in agent.invoke(thread_id=thread.id):
                        if content.role != AuthorRole.TOOL:
                            print(f"# Agent: {content.content}")

                    
                    messages = await client.agents.list_messages(thread_id=thread.id)

                    # OpenAIPageableListOfThreadMessage
                    # OpenAIPageableListOfThreadMessage


                    for file_path_annotation in messages.file_path_annotations:

                            file_name = os.path.basename(file_path_annotation.text)

                            await client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./trip")

                    
            finally:
                await client.agents.delete_thread(thread.id)
                await client.agents.delete_agent(agent.id)


        return "Saved"

# 說明：
此程式碼設置了 Azure AI 代理，用於根據使用者輸入處理預訂航班和酒店，並保存旅行計劃。它使用 Azure 憑據來創建和配置代理，通過群組聊天處理使用者輸入，並確保在任務完成後進行適當的清理。這些代理使用特定的插件（BookingPlugin 和 SavePlugin）來執行各自的任務。


In [ ]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):
    BOOKING_AGENT_NAME = "BookingAgent"
    BOOKING_AGENT_INSTRUCTIONS = """
    You are a booking agent. Help me book flights or hotels.

    Thought: Please understand the user's intention and confirm whether to use the reservation system to complete the task.

    Actions:
    - For flight bookings, convert the departure and destination names into airport codes.
    - Use the appropriate API for hotel or flight bookings. Verify that all necessary parameters are available. If any parameters are missing, ask the user to provide them. If all parameters are complete, call the corresponding function.
    - If the task is not related to hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
      - For flight bookings, output separate outbound and return contents in the order of:
        Departure Airport | Airline | Flight Number | Departure Time | Arrival Airport | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
      - For hotel bookings, output in the order of:
        Property Name | Property Description | Check-in Time | Check-out Time | Prices | Nearby Places | Hotel Class | GPS Coordinates.
    """

    SAVE_AGENT_NAME = "SaveAgent"
    SAVE_AGENT_INSTRUCTIONS = """
    You are a save tool agent. Help me to save the trip plan.
    """

    # Create agent definition
    booking_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=BOOKING_AGENT_NAME,
        instructions=BOOKING_AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent
    booking_agent = AzureAIAgent(
        client=client,
        definition=booking_agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the sample plugin to the kernel
    booking_agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create agent definition
    save_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=SAVE_AGENT_NAME,
        instructions=SAVE_AGENT_INSTRUCTIONS
    )

    # Create the AzureAI Agent
    save_agent = AzureAIAgent(
        client=client,
        definition=save_agent_definition,
    )

    save_agent.kernel.add_plugin(SavePlugin(), plugin_name="saving")

    user_inputs = [
        "I have a business trip from London to New York in Feb 20 2025 to Feb 27 2025 ,help me to book a hotel and fight tickets and save it"
    ]

    chat = AgentGroupChat(
        agents=[booking_agent, save_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[save_agent], maximum_iterations=10),
    )

    try:
        for user_input in user_inputs:
            # Add the user input as a chat message
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

            print(f"# IS COMPLETE: {chat.is_complete}")

            print("*" * 60)
            print("Chat History (In Descending Order):\n")
            async for message in chat.get_chat_messages(agent=save_agent):
                print(f"# {message.role} - {message.name or '*'}: '{message.content}'")
    finally:
        await chat.reset()
        await client.agents.delete_agent(save_agent.id)
        await client.agents.delete_agent(booking_agent.id)



---

**免責聲明**：  
本文件已使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。儘管我們努力確保翻譯的準確性，但請注意，自動翻譯可能包含錯誤或不準確之處。原始文件的母語版本應被視為權威來源。對於關鍵資訊，建議使用專業人工翻譯。我們對因使用此翻譯而引起的任何誤解或錯誤解釋不承擔責任。
